In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2015 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2015"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,2.0,The Woman in Black: Angel of Death,Relativity Media / Hammer Films,Tom Harper (director); Jon Croker (screenplay)...,[12],NaN
1,J A N U A R Y,9.0,Taken 3,20th Century Fox,"Olivier Megaton (director); Luc Besson, Robert...",[13],NaN
2,J A N U A R Y,9.0,Let's Kill Ward's Wife,Well Go USA Entertainment,Scott Foley (director/screenplay); Patrick Wil...,[14],NaN
3,J A N U A R Y,14.0,Match,IFC Films,Stephen Belber (director/screenplay); Patrick ...,[15],NaN
4,J A N U A R Y,16.0,Blackhat,Universal Pictures / Legendary Pictures,Michael Mann (director); Morgan Davis Foehl (s...,[16],NaN
...,...,...,...,...,...,...,...
206,D E C E M B E R,25.0,The Hateful Eight,The Weinstein Company,Quentin Tarantino (director/screenplay); Samue...,NaN,[217]
207,D E C E M B E R,25.0,The Revenant,20th Century Fox / Regency Enterprises / RatPa...,Alejandro G. Iñárritu (director/screenplay); M...,NaN,[218]
208,D E C E M B E R,30.0,Anomalisa,Paramount Pictures,"Charlie Kaufman, Duke Johnson (directors); Dav...",NaN,[219]
209,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_2015 = df[['Title','Cast and crew']]

In [11]:
df_2015

,Title,Cast and crew
0,The Woman in Black: Angel of Death,Tom Harper (director); Jon Croker (screenplay)...
1,Taken 3,"Olivier Megaton (director); Luc Besson, Robert..."
2,Let's Kill Ward's Wife,Scott Foley (director/screenplay); Patrick Wil...
3,Match,Stephen Belber (director/screenplay); Patrick ...
4,Blackhat,Michael Mann (director); Morgan Davis Foehl (s...
...,...,...
206,The Hateful Eight,Quentin Tarantino (director/screenplay); Samue...
207,The Revenant,Alejandro G. Iñárritu (director/screenplay); M...
208,Anomalisa,"Charlie Kaufman, Duke Johnson (directors); Dav..."
209,NaN,NaN


In [12]:
!pip install tmdbv3api

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '703a12952f3292e9958bc4d2d904a228'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2015['genres'] = df_2015['Title'].map(lambda x: get_genre(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\40042952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2015['genres'] = df_2015['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2015

,Title,Cast and crew,genres
0,The Woman in Black: Angel of Death,Tom Harper (director); Jon Croker (screenplay)...,Thriller Horror Drama
1,Taken 3,"Olivier Megaton (director); Luc Besson, Robert...",Thriller Action
2,Let's Kill Ward's Wife,Scott Foley (director/screenplay); Patrick Wil...,Comedy
3,Match,Stephen Belber (director/screenplay); Patrick ...,Comedy Drama
4,Blackhat,Michael Mann (director); Morgan Davis Foehl (s...,Crime Thriller Action
...,...,...,...
206,The Hateful Eight,Quentin Tarantino (director/screenplay); Samue...,Drama Mystery Western
207,The Revenant,Alejandro G. Iñárritu (director/screenplay); M...,Western Drama Adventure
208,Anomalisa,"Charlie Kaufman, Duke Johnson (directors); Dav...",Animation Drama Romance Comedy
209,NaN,NaN,Drama


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2015['director_name'] = df_2015['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\3596374041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2015['director_name'] = df_2015['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [20]:
df_2015['actor_1_name'] = df_2015['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\3212463946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2015['actor_1_name'] = df_2015['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [21]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [22]:
df_2015['actor_2_name'] = df_2015['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\3179629911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2015['actor_2_name'] = df_2015['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [23]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [24]:
df_2015['actor_3_name'] = df_2015['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [25]:
df_2015

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Woman in Black: Angel of Death,Tom Harper (director); Jon Croker (screenplay)...,Thriller Horror Drama,Tom Harper,Phoebe Fox,Jeremy Irvine,Helen McCrory
1,Taken 3,"Olivier Megaton (director); Luc Besson, Robert...",Thriller Action,Olivier Megaton,Liam Neeson,Forest Whitaker,Famke Janssen
2,Let's Kill Ward's Wife,Scott Foley (director/screenplay); Patrick Wil...,Comedy,Scott Foley,Patrick Wilson,Scott Foley,Donald Faison
3,Match,Stephen Belber (director/screenplay); Patrick ...,Comedy Drama,Stephen Belber,Patrick Stewart,Carla Gugino,Matthew Lillard
4,Blackhat,Michael Mann (director); Morgan Davis Foehl (s...,Crime Thriller Action,Michael Mann,Chris Hemsworth,Tang Wei,Viola Davis
...,...,...,...,...,...,...,...
206,The Hateful Eight,Quentin Tarantino (director/screenplay); Samue...,Drama Mystery Western,Quentin Tarantino,Samuel L. Jackson,Kurt Russell,Jennifer Jason Leigh
207,The Revenant,Alejandro G. Iñárritu (director/screenplay); M...,Western Drama Adventure,Alejandro G. Iñárritu,Leonardo DiCaprio,Tom Hardy,NaN
208,Anomalisa,"Charlie Kaufman, Duke Johnson (directors); Dav...",Animation Drama Romance Comedy,"Charlie Kaufman, Duke Johnson",Charlie Kaufman,Duke Johnson (directors); David Thewlis,Jennifer Jason Leigh
209,NaN,NaN,Drama,nan,nan,NaN,NaN


In [26]:
df_2015 = df_2015.rename(columns={'Title':'movie_title'})

In [27]:
new_df15 = df_2015.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [28]:
new_df15

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Tom Harper,Phoebe Fox,Jeremy Irvine,Helen McCrory,Thriller Horror Drama,The Woman in Black: Angel of Death
1,Olivier Megaton,Liam Neeson,Forest Whitaker,Famke Janssen,Thriller Action,Taken 3
2,Scott Foley,Patrick Wilson,Scott Foley,Donald Faison,Comedy,Let's Kill Ward's Wife
3,Stephen Belber,Patrick Stewart,Carla Gugino,Matthew Lillard,Comedy Drama,Match
4,Michael Mann,Chris Hemsworth,Tang Wei,Viola Davis,Crime Thriller Action,Blackhat
...,...,...,...,...,...,...
206,Quentin Tarantino,Samuel L. Jackson,Kurt Russell,Jennifer Jason Leigh,Drama Mystery Western,The Hateful Eight
207,Alejandro G. Iñárritu,Leonardo DiCaprio,Tom Hardy,NaN,Western Drama Adventure,The Revenant
208,"Charlie Kaufman, Duke Johnson",Charlie Kaufman,Duke Johnson (directors); David Thewlis,Jennifer Jason Leigh,Animation Drama Romance Comedy,Anomalisa
209,nan,nan,NaN,NaN,Drama,NaN


In [29]:
new_df15['comb'] = new_df15['actor_1_name'] + ' ' + new_df15['actor_2_name'] + ' '+ new_df15['actor_3_name'] + ' '+ new_df15['director_name'] +' ' + new_df15['genres']

In [30]:
new_df15.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      4
actor_3_name     12
genres            0
movie_title       2
comb             12
dtype: int64

In [31]:
new_df15 = new_df15.dropna(how='any')

In [32]:
new_df15.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df15['movie_title'] = new_df15['movie_title'].str.lower()

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\1572563100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df15['movie_title'] = new_df15['movie_title'].str.lower()


In [34]:
new_df15

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Tom Harper,Phoebe Fox,Jeremy Irvine,Helen McCrory,Thriller Horror Drama,the woman in black: angel of death,Phoebe Fox Jeremy Irvine Helen McCrory Tom Har...
1,Olivier Megaton,Liam Neeson,Forest Whitaker,Famke Janssen,Thriller Action,taken 3,Liam Neeson Forest Whitaker Famke Janssen Oliv...
2,Scott Foley,Patrick Wilson,Scott Foley,Donald Faison,Comedy,let's kill ward's wife,Patrick Wilson Scott Foley Donald Faison Scott...
3,Stephen Belber,Patrick Stewart,Carla Gugino,Matthew Lillard,Comedy Drama,match,Patrick Stewart Carla Gugino Matthew Lillard S...
4,Michael Mann,Chris Hemsworth,Tang Wei,Viola Davis,Crime Thriller Action,blackhat,Chris Hemsworth Tang Wei Viola Davis Michael M...
...,...,...,...,...,...,...,...
203,Sean Anders,Will Ferrell,Mark Wahlberg,Linda Cardellini,Comedy,daddy's home,Will Ferrell Mark Wahlberg Linda Cardellini Se...
204,David O. Russell,Jennifer Lawrence,Robert De Niro,Édgar Ramírez,Drama Comedy,joy,Jennifer Lawrence Robert De Niro Édgar Ramírez...
205,Ericson Core,Édgar Ramírez,Luke Bracey,Teresa Palmer,Action Thriller Crime,point break,Édgar Ramírez Luke Bracey Teresa Palmer Ericso...
206,Quentin Tarantino,Samuel L. Jackson,Kurt Russell,Jennifer Jason Leigh,Drama Mystery Western,the hateful eight,Samuel L. Jackson Kurt Russell Jennifer Jason ...


In [35]:
old_df = pd.read_csv('final_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [37]:
final_df = old_df.append(new_df15,ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_17124\2634544481.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df15,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6063,Sean Anders,Will Ferrell,Mark Wahlberg,Linda Cardellini,Comedy,daddy's home,Will Ferrell Mark Wahlberg Linda Cardellini Se...
6064,David O. Russell,Jennifer Lawrence,Robert De Niro,Édgar Ramírez,Drama Comedy,joy,Jennifer Lawrence Robert De Niro Édgar Ramírez...
6065,Ericson Core,Édgar Ramírez,Luke Bracey,Teresa Palmer,Action Thriller Crime,point break,Édgar Ramírez Luke Bracey Teresa Palmer Ericso...
6066,Quentin Tarantino,Samuel L. Jackson,Kurt Russell,Jennifer Jason Leigh,Drama Mystery Western,the hateful eight,Samuel L. Jackson Kurt Russell Jennifer Jason ...


In [39]:
final_df.to_csv('main_data.csv',index=False)